In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
from scipy.constants import Stefan_Boltzmann
import xarray as xr

import matplotlib.pyplot as plt

import synthia as syn
from synthia.util import to_stacked_array, to_unstacked_dataset

## Input data

In [ ]:
THIS_DIR = Path.cwd()
ds_input = xr.open_dataset(THIS_DIR.parents[1] / 'data' / 'nwp_saf_profiles_in.nc')

input_relevant = [
    'temperature_fl',
    'temperature_hl',
    'pressure_hl',
]

ds_true_in = ds_input[input_relevant]
ds_true_in

## Physical model

In [ ]:
def compute_optical_depth(ds: xr.Dataset) -> xr.DataArray:
    p_hl_a = ds['pressure_hl'].sel(half_level=slice(1,None))
    p_hl_b = ds['pressure_hl'].sel(half_level=slice(None,-1))
    delta_pressure = p_hl_a - p_hl_b 
    delta_pressure = delta_pressure.rename({'half_level': 'level'})
    g = 9.80665 # m/s²
    rho_liquid = 1000 # kg/m³
    rho_ice = 917 # kg/m³
    optical_depth = (ds['q_liquid'] / (rho_liquid * ds['re_liquid']) +\
                     ds['q_ice'] / (rho_ice * ds['re_ice']) ) * delta_pressure / g
    optical_depth = optical_depth.rename('optical_depth_fl')
    return xr.merge([ds, optical_depth])

In [ ]:
def calc_emiss(sigma_hl, total_column_opt_depth, layer_cloud_optical_depth):
    """ Calculate per layer emissivity
    """
    diffusivity_factor = 1/np.cos(np.radians(53))
    layer_gas_opt_depth = total_column_opt_depth * sigma_hl.diff('half_level').rename({'half_level': 'level'})
    layer_opt_depths = layer_gas_opt_depth + layer_cloud_optical_depth
    emiss = 1 - np.exp(-diffusivity_factor * layer_opt_depths)
    return emiss

def calc_plank(temperature):
    b = Stefan_Boltzmann * temperature**4
    return b

def calc_lw_from_temperature(ds, total_column_opt_depth, layer_cloud_optical_depth):
    flux_dn_hl = xr.zeros_like(ds.pressure_hl).rename('flux_dn_lw')
    flux_dn_hl.attrs = {'long_name': 'Downward logwave flux', 'units': 'W/m2'}
    
    plank_fl = Stefan_Boltzmann * ds['temperature_fl']**4
    sigma_hl = ds_input['pressure_hl'] / ds_input['pressure_hl'].sel(half_level=-1)
    emiss_fl = calc_emiss(sigma_hl, total_column_opt_depth, layer_cloud_optical_depth)
    
    # BC @ TOA: no flux
    flux_dn_hl[:, 0] = 0.

    for half_level in range(flux_dn_hl.shape[1]-1):
        flux_dn_hl[:, half_level+1] = flux_dn_hl[:, half_level] * (1 - emiss_fl[:, half_level]) + \
                                 plank_fl[:, half_level] * emiss_fl[:, half_level]
    return xr.merge([ds, flux_dn_hl])

In [ ]:
layer_cloud_optical_depth = compute_optical_depth(ds_input)['optical_depth_fl']

In [ ]:
total_column_opt_depth = 1.7 # We try with 30, range is 0.5 to many thousands.
simple_out = calc_lw_from_temperature(ds_true_in, total_column_opt_depth, layer_cloud_optical_depth)

N_COLUMNS = 10
fig, ax = plt.subplots(N_COLUMNS,1, figsize=(6, 4*N_COLUMNS))
for i, column in enumerate(np.random.choice(simple_out.column, N_COLUMNS)):
    simple_out.flux_dn_lw.sel(column=column).plot(ax=ax[i], label='Simple')
    #sparta_out.flux_dn_lw.sel(column=column).plot(ax=ax[i], label='Spartacus')
    #triple_out.flux_dn_lw.sel(column=column).plot(ax=ax[i], label='Tripleclouds')
    ax[i].legend()